In [333]:
import sqlalchemy
import pandas as pd
from geopy.geocoders import MapBox
from geopy.distance import geodesic

In [334]:
engine = sqlalchemy.create_engine('sqlite:///db/ncaa_history.sqlite')
connection = engine.connect()

## Import SQL Database Tables

In [335]:
games_df = pd.read_sql_table('games', engine)
games_df.head()

,location,loser,loser_score,loser_seed,round,winner,winner_score,winner_seed,year
0,"Pittsburgh, PA",Radford,61,16,1,Villanova,87,1,2018
1,"Pittsburgh, PA",Virginia Tech,83,8,1,Alabama,86,9,2018
2,"San Diego, CA",Murray State,68,12,1,West Virginia,85,5,2018
3,"San Diego, CA",Wichita State,75,4,1,Marshall,81,13,2018
4,"Dallas, TX",St. Bonaventure,62,11,1,Florida,77,6,2018


In [336]:
games_df = df[['year', 'round', 'location', 'winner', 'winner_seed', 'winner_score', 'loser', 'loser_seed', 'loser_score']]
games_df.head()

,year,round,location,winner,winner_seed,winner_score,loser,loser_seed,loser_score
0,2018,1,"Pittsburgh, PA",Villanova,1,87,Radford,16,61
1,2018,1,"Pittsburgh, PA",Alabama,9,86,Virginia Tech,8,83
2,2018,1,"San Diego, CA",West Virginia,5,85,Murray State,12,68
3,2018,1,"San Diego, CA",Marshall,13,81,Wichita State,4,75
4,2018,1,"Dallas, TX",Florida,6,77,St. Bonaventure,11,62


In [337]:
schools_df = pd.read_sql_table('schools', engine)
schools_df.head()

,location,logo,name,url
0,"Abilene, Texas",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Abilene Christian Wildcats,https://www.sports-reference.com/cbb/schools/a...
1,"USAF Academy, Colorado",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Air Force,https://www.sports-reference.com/cbb/schools/a...
2,"Akron, Ohio",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Akron,https://www.sports-reference.com/cbb/schools/a...
3,"Normal, Alabama",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Alabama A&M Bulldogs,https://www.sports-reference.com/cbb/schools/a...
4,"Tuscaloosa, Alabama",https://d2p3bygnnzw9w3.cloudfront.net/req/2019...,Alabama,https://www.sports-reference.com/cbb/schools/a...


In [338]:
school_locations = schools_df.loc[:,['location','name']]
school_locations.head()

,location,name
0,"Abilene, Texas",Abilene Christian Wildcats
1,"USAF Academy, Colorado",Air Force
2,"Akron, Ohio",Akron
3,"Normal, Alabama",Alabama A&M Bulldogs
4,"Tuscaloosa, Alabama",Alabama


In [339]:
teams_df = pd.read_sql_table('teams', engine)
teams_df.head()

,name,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,seed,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,url,year
0,Villanova,40,8075,532,105,928,1028,756,.490,1543,272,.317,858,.428,2401,479,.747,641,378,698,2807,70.2,193,512,1306,,,335,148,328,346,346,151,346,265,21,332,107,350,241,334,204,301,24,346,114,138,31,327,1,10.24,26.64,655,162,1056,1220,756,.590,1282,464,.401,1158,.500,2440,559,.779,718,40,8075,380,645,3463,86.6,10,1,3,31,4,1,16,4,96,1,13,2,5,1,32,10,73,,,80,273,1,1,28,182,8,259,426,1436,https://www.sports-reference.com/cbb/schools/v...,2018
1,Radford,36,7325,413,120,818,819,558,.484,1152,261,.350,745,.432,1897,437,.751,582,317,601,2336,64.9,198,469,1135,,,116,257,177,149,117,131,132,217,188,217,128,172,157,339,113,150,164,149,20,160,87,165,16,-4.08,-2.61,442,105,830,850,573,.475,1207,277,.349,793,.425,2000,445,.725,614,36,7325,414,596,2422,67.3,265,191,185,184,171,175,219,287,167,106,178,87,285,106,199,132,214,,,32,196,169,312,97,223,93,226,441,1244,https://www.sports-reference.com/cbb/schools/r...,2018
2,Virginia Tech,33,6675,460,124,784,844,552,.492,1123,292,.347,842,.430,1965,402,.697,577,346,572,2382,72.2,174,433,1130,,,236,274,113,197,110,159,106,316,165,323,113,242,83,76,108,230,222,189,169,49,160,158,8,6.71,14.35,542,74,834,931,642,.573,1120,289,.388,745,.499,1865,483,.709,681,33,6675,244,523,2634,79.8,66,34,37,310,163,80,104,6,250,82,30,139,6,214,130,190,121,,,326,40,75,46,213,92,262,191,395,1078,https://www.sports-reference.com/cbb/schools/v...,2018
3,Alabama,36,7200,427,134,863,877,615,.469,1310,262,.328,799,.416,2109,527,.685,769,414,718,2543,70.6,237,473,1277,,,150,308,255,237,230,65,291,228,57,295,50,309,306,33,321,336,12,283,130,323,83,316,9,10.64,12.34,458,192,932,910,681,.535,1273,229,.326,702,.461,1975,555,.670,828,36,7200,358,676,2604,72.3,6,49,149,11,51,98,60,58,106,233,298,187,100,124,34,310,11,,,118,311,90,217,90,346,61,228,513,1290,https://www.sports-reference.com/cbb/schools/a...,2018
4,West Virginia,37,7425,462,148,893,835,547,.454,1204,288,.380,758,.426,1962,617,.712,866,375,695,2575,69.6,204,612,1268,,,245,339,307,177,95,26,200,306,323,239,90,238,349,183,349,297,29,291,100,209,3,308,5,9.36,19.88,561,192,888,1036,714,.488,1463,322,.353,911,.436,2374,570,.766,744,37,7425,513,798,2964,80.1,26,9,22,11,89,14,36,237,8,37,156,24,231,3,25,25,56,,,1,351,13,45,5,161,13,301,420,1401,https://www.sports-reference.com/cbb/schools/w...,2018


## School/Game Locations

In [340]:
# merge games table and school locations to start games_stats table
game_stats = games_df.merge(school_locations, how='left', left_on='winner', right_on='name', suffixes = ('','_winner'))
game_stats = game_stats.merge(school_locations, how='left', left_on='loser', right_on='name', suffixes = ('','_loser'))
del game_stats['name']
del game_stats['name_loser']
game_stats.head()

,year,round,location,winner,winner_seed,winner_score,loser,loser_seed,loser_score,location_winner,location_loser
0,2018,1,"Pittsburgh, PA",Villanova,1,87,Radford,16,61,"Villanova, Pennsylvania","Radford, Virginia"
1,2018,1,"Pittsburgh, PA",Alabama,9,86,Virginia Tech,8,83,"Tuscaloosa, Alabama","Blacksburg, Virginia"
2,2018,1,"San Diego, CA",West Virginia,5,85,Murray State,12,68,"Morgantown, West Virginia","Murray, Kentucky"
3,2018,1,"San Diego, CA",Marshall,13,81,Wichita State,4,75,"Huntington, West Virginia","Wichita, Kansas"
4,2018,1,"Dallas, TX",Florida,6,77,St. Bonaventure,11,62,"Gainesville, Florida","St. Bonaventure, New York"


In [341]:
geolocator = MapBox(api_key="pk.eyJ1Ijoib2JuaWNob2xzb24iLCJhIjoiY2pwcHBrbmIxMGdhMTN4cWZ2czR6NDVwcCJ9.T5lnDc1uaxKgp4S18rFyBw", timeout=None)

In [342]:
def get_latlons(column):
    counter = 0
    locations = []
    latlons = []

    for location in column:
        geolocation = geolocator.geocode(location)
        locations.append(geolocation)
        counter +=1
        if (counter % 100 == 0):
            print(counter)

    for location in locations:
        latlon = (location.latitude, location.longitude)
        latlons.append(latlon)
    
    return latlons

In [343]:
game_locations = get_latlons(game_stats['location'])
winner_locations = get_latlons(game_stats['location_winner'])
loser_locations = get_latlons(game_stats['location_loser'])

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800


In [362]:
def get_travel_distances(school_location_list):
    travel_distances = []
    
    for index, school_location in enumerate(school_location_list):
        game_location = game_locations[index]
        travel_distance = geodesic(school_location, game_location).miles
        travel_distances.append(travel_distance)
        
    return travel_distances

In [363]:
winner_travel_distances = get_travel_distances(winner_locations)
loser_travel_distances = get_travel_distances(loser_locations)

In [365]:
pd.options.mode.chained_assignment = None

game_stats['travel_miles_winner'] = winner_travel_distances
game_stats['travel_miles_loser'] = loser_travel_distances
game_stats.head()

,year,round,location,winner,winner_seed,winner_score,loser,loser_seed,loser_score,location_winner,location_loser,travel_miles_winner,travel_miles_loser
0,2018,1,"Pittsburgh, PA",Villanova,1,87,Radford,16,61,"Villanova, Pennsylvania","Radford, Virginia",246.946684,230.496683
1,2018,1,"Pittsburgh, PA",Alabama,9,86,Virginia Tech,8,83,"Tuscaloosa, Alabama","Blacksburg, Virginia",645.478758,222.738124
2,2018,1,"San Diego, CA",West Virginia,5,85,Murray State,12,68,"Morgantown, West Virginia","Murray, Kentucky",2117.874565,1658.405787
3,2018,1,"San Diego, CA",Marshall,13,81,Wichita State,4,75,"Huntington, West Virginia","Wichita, Kansas",1982.540076,1170.375993
4,2018,1,"Dallas, TX",Florida,6,77,St. Bonaventure,11,62,"Gainesville, Florida","St. Bonaventure, New York",882.710411,1190.429282


## Create Losing/Winning Records with Season Stats

In [367]:
winners_df = game_stats.rename(columns={'winner':'team_name', 'winner_seed':'team_seed', 'winner_score':'team_score', 
                                       'loser':'opponent_name', 'loser_seed':'opponent_seed', 'loser_score':'opponent_score', 
                                       'location_winner':'team_location', 'location_loser':'opponent_location', 
                                       'travel_miles_winner':'team_travel_miles', 'travel_miles_loser':'opponent_travel_miles'})
winners_df['outcome'] = 'WIN'
winners_df = winners_df[['year', 'round', 'location', 'team_name', 'team_seed', 'team_score', 'team_location', 
                         'team_travel_miles', 'opponent_name', 'opponent_seed', 'opponent_score', 'opponent_location', 
                         'opponent_travel_miles', 'outcome']]
winners_df.head()

,year,round,location,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,outcome
0,2018,1,"Pittsburgh, PA",Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,61,"Radford, Virginia",230.496683,WIN
1,2018,1,"Pittsburgh, PA",Alabama,9,86,"Tuscaloosa, Alabama",645.478758,Virginia Tech,8,83,"Blacksburg, Virginia",222.738124,WIN
2,2018,1,"San Diego, CA",West Virginia,5,85,"Morgantown, West Virginia",2117.874565,Murray State,12,68,"Murray, Kentucky",1658.405787,WIN
3,2018,1,"San Diego, CA",Marshall,13,81,"Huntington, West Virginia",1982.540076,Wichita State,4,75,"Wichita, Kansas",1170.375993,WIN
4,2018,1,"Dallas, TX",Florida,6,77,"Gainesville, Florida",882.710411,St. Bonaventure,11,62,"St. Bonaventure, New York",1190.429282,WIN


In [368]:
losers_df = game_stats.rename(columns={'winner':'opponent_name', 'winner_seed':'opponent_seed', 'winner_score':'opponent_score', 
                                       'loser':'team_name', 'loser_seed':'team_seed', 'loser_score':'team_score', 
                                       'location_winner':'opponent_location', 'location_loser':'team_location', 
                                       'travel_miles_winner':'opponent_travel_miles', 'travel_miles_loser':'team_travel_miles'})
losers_df['outcome'] = 'LOSS'
losers_df = losers_df[['year', 'round', 'location', 'team_name', 'team_seed', 'team_score', 'team_location', 
                         'team_travel_miles', 'opponent_name', 'opponent_seed', 'opponent_score', 'opponent_location', 
                         'opponent_travel_miles', 'outcome']]
losers_df.head()

,year,round,location,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,outcome
0,2018,1,"Pittsburgh, PA",Radford,16,61,"Radford, Virginia",230.496683,Villanova,1,87,"Villanova, Pennsylvania",246.946684,LOSS
1,2018,1,"Pittsburgh, PA",Virginia Tech,8,83,"Blacksburg, Virginia",222.738124,Alabama,9,86,"Tuscaloosa, Alabama",645.478758,LOSS
2,2018,1,"San Diego, CA",Murray State,12,68,"Murray, Kentucky",1658.405787,West Virginia,5,85,"Morgantown, West Virginia",2117.874565,LOSS
3,2018,1,"San Diego, CA",Wichita State,4,75,"Wichita, Kansas",1170.375993,Marshall,13,81,"Huntington, West Virginia",1982.540076,LOSS
4,2018,1,"Dallas, TX",St. Bonaventure,11,62,"St. Bonaventure, New York",1190.429282,Florida,6,77,"Gainesville, Florida",882.710411,LOSS


In [369]:
dataframes = [winners_df, losers_df]
outcomes_df = pd.concat(dataframes, sort=False)
print(len(winners_df), len(losers_df), len(outcomes_df))

1872 1872 3744


In [370]:
outcomes_incl_team_stats = outcomes_df.merge(teams_df, how='left', left_on=['team_name','year'], right_on=['name','year'], suffixes=('','_stats'))
outcomes_full_stats = outcomes_incl_team_stats.merge(teams_df.add_prefix('opponent_'), how='left', left_on=['opponent_name','year'], right_on=['opponent_name','opponent_year'], suffixes=('','_stats'))
outcomes_full_stats.head()

,year,round,location,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,outcome,name,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,seed,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,url,opponent_opp_g,opponent_opp_mp,opponent_opp_opp_ast,opponent_opp_opp_blk,opponent_opp_opp_drb,opponent_opp_opp_fg,opponent_opp_opp_fg2,opponent_opp_opp_fg2_pct,opponent_opp_opp_fg2a,opponent_opp_opp_fg3,opponent_opp_opp_fg3_pct,opponent_opp_opp_fg3a,opponent_opp_opp_fg_pct,opponent_opp_opp_fga,opponent_opp_opp_ft,opponent_opp_opp_ft_pct,opponent_opp_opp_fta,opponent_opp_opp_orb,opponent_opp_opp_pf,opponent_opp_opp_pts,opponent_opp_opp_pts_per_g,opponent_opp_opp_stl,opponent_opp_opp_tov,opponent_opp_opp_trb,opponent_opp_rank_g,opponent_opp_rank_mp,opponent_opp_rank_opp_ast,opponent_opp_rank_opp_blk,opponent_opp_rank_opp_drb,opponent_opp_rank_opp_fg,opponent_opp_rank_opp_fg2,opponent_opp_rank_opp_fg2_pct,opponent_opp_rank_opp_fg2a,opponent_opp_rank_opp_fg3,opponent_opp_rank_opp_fg3_pct,opponent_opp_rank_opp_fg3a,opponent_opp_rank_opp_fg_pct,opponent_opp_rank_opp_fga,opponent_opp_rank_opp_ft,opponent_opp_rank_opp_ft_pct,opponent_opp_rank_opp_fta,opponent_opp_rank_opp_orb,opponent_opp_rank_opp_pf,opponent_opp_rank_opp_pts,opponent_opp_rank_opp_pts_per_g,opponent_opp_rank_opp_stl,opponent_opp_rank_opp_tov,opponent_opp_rank_opp_trb,opponent_seed_stats,opponent_team_SOS,opponent_team_SRS,opponent_team_ast,opponent_team_blk,opponent_team_drb,opponent_team_fg,opponent_team_fg2,opponent_team_fg2_pct,opponent_team_fg2a,opponent_team_fg3,opponent_team_fg3_pct,opponent_team_fg3a,opponent_team_fg_pct,opponent_team_fga,opponent_team_ft,opponent_team_ft_pct,opponent_team_fta,opponent_team_g,opponent_team_mp,opponent_team_orb,opponent_team_pf,opponent_team_pts,opponent_team_pts_per_g,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_drb,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_g,opponent_team_rank_mp,opponent_team_rank_orb,opponent_team_rank_pf,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_url,opponent_year
0,2018,1,"Pittsburgh, PA",Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,61,"Radford, Virginia",230.496683,WIN,Villanova,40,8075,532,105,928,1028,756,.490,1543,272,

In [371]:
list(outcomes_full_stats.columns.values)

['year',
 'round',
 'location',
 'team_name',
 'team_seed',
 'team_score',
 'team_location',
 'team_travel_miles',
 'opponent_name',
 'opponent_seed',
 'opponent_score',
 'opponent_location',
 'opponent_travel_miles',
 'outcome',
 'name',
 'opp_g',
 'opp_mp',
 'opp_opp_ast',
 'opp_opp_blk',
 'opp_opp_drb',
 'opp_opp_fg',
 'opp_opp_fg2',
 'opp_opp_fg2_pct',
 'opp_opp_fg2a',
 'opp_opp_fg3',
 'opp_opp_fg3_pct',
 'opp_opp_fg3a',
 'opp_opp_fg_pct',
 'opp_opp_fga',
 'opp_opp_ft',
 'opp_opp_ft_pct',
 'opp_opp_fta',
 'opp_opp_orb',
 'opp_opp_pf',
 'opp_opp_pts',
 'opp_opp_pts_per_g',
 'opp_opp_stl',
 'opp_opp_tov',
 'opp_opp_trb',
 'opp_rank_g',
 'opp_rank_mp',
 'opp_rank_opp_ast',
 'opp_rank_opp_blk',
 'opp_rank_opp_drb',
 'opp_rank_opp_fg',
 'opp_rank_opp_fg2',
 'opp_rank_opp_fg2_pct',
 'opp_rank_opp_fg2a',
 'opp_rank_opp_fg3',
 'opp_rank_opp_fg3_pct',
 'opp_rank_opp_fg3a',
 'opp_rank_opp_fg_pct',
 'opp_rank_opp_fga',
 'opp_rank_opp_ft',
 'opp_rank_opp_ft_pct',
 'opp_rank_opp_fta',
 'opp_ran

In [372]:
model_input_df = outcomes_full_stats[['year', 'round', 'location', 'outcome', 'team_name', 'team_seed', 'team_score', 
                                     'team_location', 'team_travel_miles', 'opponent_name', 'opponent_seed', 
                                     'opponent_score', 'opponent_location', 'opponent_travel_miles', 'team_SOS', 
                                     'team_SRS', 'team_ast', 'team_blk', 'team_drb', 'team_fg', 'team_fg2',
                                      'team_fg2_pct', 'team_fg2a', 'team_fg3', 'team_fg3_pct', 'team_fg3a', 
                                      'team_fg_pct', 'team_fga', 'team_ft', 'team_ft_pct', 'team_fta', 'team_g', 
                                      'team_mp', 'team_orb', 'team_pf', 'team_pts', 'team_pts_per_g', 'team_rank_SOS', 
                                      'team_rank_SRS', 'team_rank_ast', 'team_rank_blk', 'team_rank_drb', 'team_rank_fg', 
                                      'team_rank_fg2', 'team_rank_fg2_pct', 'team_rank_fg2a', 'team_rank_fg3', 
                                      'team_rank_fg3_pct', 'team_rank_fg3a', 'team_rank_fg_pct', 'team_rank_fga', 
                                      'team_rank_ft', 'team_rank_ft_pct', 'team_rank_fta', 'team_rank_g', 'team_rank_mp', 
                                      'team_rank_orb', 'team_rank_pf', 'team_rank_pts', 'team_rank_pts_per_g', 
                                      'team_rank_stl', 'team_rank_tov', 'team_rank_trb', 'team_stl', 'team_tov', 
                                      'team_trb', 'opp_g', 'opp_mp', 'opp_opp_ast', 'opp_opp_blk', 'opp_opp_drb', 
                                      'opp_opp_fg', 'opp_opp_fg2', 'opp_opp_fg2_pct', 'opp_opp_fg2a', 'opp_opp_fg3', 
                                      'opp_opp_fg3_pct', 'opp_opp_fg3a', 'opp_opp_fg_pct', 'opp_opp_fga', 'opp_opp_ft', 
                                      'opp_opp_ft_pct', 'opp_opp_fta', 'opp_opp_orb', 'opp_opp_pf', 'opp_opp_pts',
                                      'opp_opp_pts_per_g', 'opp_opp_stl', 'opp_opp_tov', 'opp_opp_trb', 'opp_rank_g',
                                      'opp_rank_mp', 'opp_rank_opp_ast', 'opp_rank_opp_blk', 'opp_rank_opp_drb',
                                      'opp_rank_opp_fg', 'opp_rank_opp_fg2', 'opp_rank_opp_fg2_pct', 'opp_rank_opp_fg2a', 
                                      'opp_rank_opp_fg3', 'opp_rank_opp_fg3_pct', 'opp_rank_opp_fg3a','opp_rank_opp_fg_pct', 
                                      'opp_rank_opp_fga', 'opp_rank_opp_ft', 'opp_rank_opp_ft_pct', 'opp_rank_opp_fta',
                                      'opp_rank_opp_orb', 'opp_rank_opp_pf', 'opp_rank_opp_pts', 'opp_rank_opp_pts_per_g',
                                      'opp_rank_opp_stl', 'opp_rank_opp_tov', 'opp_rank_opp_trb',  'opponent_team_SOS', 
                                      'opponent_team_SRS', 'opponent_team_ast', 'opponent_team_blk', 'opponent_team_drb', 
                                      'opponent_team_fg', 'opponent_team_fg2', 'opponent_team_fg2_pct', 'opponent_team_fg2a', 
                                      'opponent_team_fg3', 'opponent_team_fg3_pct', 'opponent_team_fg3a', 'opponent_team_fg_pct', 
                                      'opponent_team_fga', 'opponent_team_ft', 'opponent_team_ft_pct', 'opponent_team_fta', 
                                      'opponent_team_g', 'opponent_team_mp', 'opponent_team_orb', 'opponent_team_pf', 
                                      'opponent_team_pts', 'opponent_team_pts_per_g', 'opponent_team_rank_SOS', 
                                      'opponent_team_rank_SRS', 'opponent_team_rank_ast', 'opponent_team_rank_blk', 
                                      'opponent_team_rank_drb', 'opponent_team_rank_fg', 'opponent_team_rank_fg2', 
                                      'opponent_team_rank_fg2_pct', 'opponent_team_rank_fg2a', 'opponent_team_rank_fg3', 
                                      'opponent_team_rank_fg3_pct', 'opponent_team_rank_fg3a', 'opponent_team_rank_fg_pct', 
                                      'opponent_team_rank_fga', 'opponent_team_rank_ft', 'opponent_team_rank_ft_pct', 
                                      'opponent_team_rank_fta', 'opponent_team_rank_g', 'opponent_team_rank_mp', 
                                      'opponent_team_rank_orb', 'opponent_team_rank_pf', 'opponent_team_rank_pts', 
                                      'opponent_team_rank_pts_per_g', 'opponent_team_rank_stl', 'opponent_team_rank_tov', 
                                      'opponent_team_rank_trb', 'opponent_team_stl', 'opponent_team_tov', 'opponent_team_trb', 
                                      'opponent_opp_g', 'opponent_opp_mp', 'opponent_opp_opp_ast', 'opponent_opp_opp_blk', 
                                      'opponent_opp_opp_drb', 'opponent_opp_opp_fg', 'opponent_opp_opp_fg2', 
                                      'opponent_opp_opp_fg2_pct', 'opponent_opp_opp_fg2a', 'opponent_opp_opp_fg3', 
                                      'opponent_opp_opp_fg3_pct', 'opponent_opp_opp_fg3a', 'opponent_opp_opp_fg_pct', 
                                      'opponent_opp_opp_fga', 'opponent_opp_opp_ft', 'opponent_opp_opp_ft_pct', 
                                      'opponent_opp_opp_fta', 'opponent_opp_opp_orb', 'opponent_opp_opp_pf', 
                                      'opponent_opp_opp_pts', 'opponent_opp_opp_pts_per_g', 'opponent_opp_opp_stl', 
                                      'opponent_opp_opp_tov', 'opponent_opp_opp_trb', 'opponent_opp_rank_g', 
                                      'opponent_opp_rank_mp', 'opponent_opp_rank_opp_ast', 'opponent_opp_rank_opp_blk', 
                                      'opponent_opp_rank_opp_drb', 'opponent_opp_rank_opp_fg', 'opponent_opp_rank_opp_fg2', 
                                      'opponent_opp_rank_opp_fg2_pct', 'opponent_opp_rank_opp_fg2a', 'opponent_opp_rank_opp_fg3', 
                                      'opponent_opp_rank_opp_fg3_pct', 'opponent_opp_rank_opp_fg3a', 'opponent_opp_rank_opp_fg_pct', 
                                      'opponent_opp_rank_opp_fga', 'opponent_opp_rank_opp_ft', 'opponent_opp_rank_opp_ft_pct', 
                                      'opponent_opp_rank_opp_fta', 'opponent_opp_rank_opp_orb', 'opponent_opp_rank_opp_pf', 
                                      'opponent_opp_rank_opp_pts', 'opponent_opp_rank_opp_pts_per_g', 'opponent_opp_rank_opp_stl', 
                                      'opponent_opp_rank_opp_tov', 'opponent_opp_rank_opp_trb']]

model_input_df.head()

,year,round,location,outcome,team_name,team_seed,team_score,team_location,team_travel_miles,opponent_name,opponent_seed,opponent_score,opponent_location,opponent_travel_miles,team_SOS,team_SRS,team_ast,team_blk,team_drb,team_fg,team_fg2,team_fg2_pct,team_fg2a,team_fg3,team_fg3_pct,team_fg3a,team_fg_pct,team_fga,team_ft,team_ft_pct,team_fta,team_g,team_mp,team_orb,team_pf,team_pts,team_pts_per_g,team_rank_SOS,team_rank_SRS,team_rank_ast,team_rank_blk,team_rank_drb,team_rank_fg,team_rank_fg2,team_rank_fg2_pct,team_rank_fg2a,team_rank_fg3,team_rank_fg3_pct,team_rank_fg3a,team_rank_fg_pct,team_rank_fga,team_rank_ft,team_rank_ft_pct,team_rank_fta,team_rank_g,team_rank_mp,team_rank_orb,team_rank_pf,team_rank_pts,team_rank_pts_per_g,team_rank_stl,team_rank_tov,team_rank_trb,team_stl,team_tov,team_trb,opp_g,opp_mp,opp_opp_ast,opp_opp_blk,opp_opp_drb,opp_opp_fg,opp_opp_fg2,opp_opp_fg2_pct,opp_opp_fg2a,opp_opp_fg3,opp_opp_fg3_pct,opp_opp_fg3a,opp_opp_fg_pct,opp_opp_fga,opp_opp_ft,opp_opp_ft_pct,opp_opp_fta,opp_opp_orb,opp_opp_pf,opp_opp_pts,opp_opp_pts_per_g,opp_opp_stl,opp_opp_tov,opp_opp_trb,opp_rank_g,opp_rank_mp,opp_rank_opp_ast,opp_rank_opp_blk,opp_rank_opp_drb,opp_rank_opp_fg,opp_rank_opp_fg2,opp_rank_opp_fg2_pct,opp_rank_opp_fg2a,opp_rank_opp_fg3,opp_rank_opp_fg3_pct,opp_rank_opp_fg3a,opp_rank_opp_fg_pct,opp_rank_opp_fga,opp_rank_opp_ft,opp_rank_opp_ft_pct,opp_rank_opp_fta,opp_rank_opp_orb,opp_rank_opp_pf,opp_rank_opp_pts,opp_rank_opp_pts_per_g,opp_rank_opp_stl,opp_rank_opp_tov,opp_rank_opp_trb,opponent_team_SOS,opponent_team_SRS,opponent_team_ast,opponent_team_blk,opponent_team_drb,opponent_team_fg,opponent_team_fg2,opponent_team_fg2_pct,opponent_team_fg2a,opponent_team_fg3,opponent_team_fg3_pct,opponent_team_fg3a,opponent_team_fg_pct,opponent_team_fga,opponent_team_ft,opponent_team_ft_pct,opponent_team_fta,opponent_team_g,opponent_team_mp,opponent_team_orb,opponent_team_pf,opponent_team_pts,opponent_team_pts_per_g,opponent_team_rank_SOS,opponent_team_rank_SRS,opponent_team_rank_ast,opponent_team_rank_blk,opponent_team_rank_drb,opponent_team_rank_fg,opponent_team_rank_fg2,opponent_team_rank_fg2_pct,opponent_team_rank_fg2a,opponent_team_rank_fg3,opponent_team_rank_fg3_pct,opponent_team_rank_fg3a,opponent_team_rank_fg_pct,opponent_team_rank_fga,opponent_team_rank_ft,opponent_team_rank_ft_pct,opponent_team_rank_fta,opponent_team_rank_g,opponent_team_rank_mp,opponent_team_rank_orb,opponent_team_rank_pf,opponent_team_rank_pts,opponent_team_rank_pts_per_g,opponent_team_rank_stl,opponent_team_rank_tov,opponent_team_rank_trb,opponent_team_stl,opponent_team_tov,opponent_team_trb,opponent_opp_g,opponent_opp_mp,opponent_opp_opp_ast,opponent_opp_opp_blk,opponent_opp_opp_drb,opponent_opp_opp_fg,opponent_opp_opp_fg2,opponent_opp_opp_fg2_pct,opponent_opp_opp_fg2a,opponent_opp_opp_fg3,opponent_opp_opp_fg3_pct,opponent_opp_opp_fg3a,opponent_opp_opp_fg_pct,opponent_opp_opp_fga,opponent_opp_opp_ft,opponent_opp_opp_ft_pct,opponent_opp_opp_fta,opponent_opp_opp_orb,opponent_opp_opp_pf,opponent_opp_opp_pts,opponent_opp_opp_pts_per_g,opponent_opp_opp_stl,opponent_opp_opp_tov,opponent_opp_opp_trb,opponent_opp_rank_g,opponent_opp_rank_mp,opponent_opp_rank_opp_ast,opponent_opp_rank_opp_blk,opponent_opp_rank_opp_drb,opponent_opp_rank_opp_fg,opponent_opp_rank_opp_fg2,opponent_opp_rank_opp_fg2_pct,opponent_opp_rank_opp_fg2a,opponent_opp_rank_opp_fg3,opponent_opp_rank_opp_fg3_pct,opponent_opp_rank_opp_fg3a,opponent_opp_rank_opp_fg_pct,opponent_opp_rank_opp_fga,opponent_opp_rank_opp_ft,opponent_opp_rank_opp_ft_pct,opponent_opp_rank_opp_fta,opponent_opp_rank_opp_orb,opponent_opp_rank_opp_pf,opponent_opp_rank_opp_pts,opponent_opp_rank_opp_pts_per_g,opponent_opp_rank_opp_stl,opponent_opp_rank_opp_tov,opponent_opp_rank_opp_trb
0,2018,1,"Pittsburgh, PA",WIN,Villanova,1,87,"Villanova, Pennsylvania",246.946684,Radford,16,61,"Radford, Virginia",230.496683,10.24,26.64,655,162,1056,1220,756,.590,1282,464,.401,1158,.500,2440,559,.779,718,40,8075,380,645,3463,86.6,10,1,3,

## Create/Train/Test/Refine Model